# Fiddling around with making Bungie API calls to get json data 

In [66]:
import pandas as pd
import requests
import os

In [181]:
class BungieData(object):

    def __init__(self, api_key, session=None):
        '''
        session: an object from login.py defining a user session by 
        logging into Bungie.net (oAuth verified)
        '''
        self.session = session
        self.api_key = api_key
        
    def get_playerByTagName(self, gamertag):
        site_call = "https://bungie.net/Platform/Destiny2/SearchDestinyPlayer/2/" + gamertag
        request = requests.get(site_call, 
                                headers={"X-API-Key":self.api_key})
        return request.json()

    def get_BungieUserId(self, membership_id):
        '''
        Use old Destiny endpoint for PSN user to get the BUNGIE membershipId for a user
        '''
        site_call = "https://bungie.net/Platform/User/GetMembershipsById/" + str(membership_id) + "/2/" 
        request = requests.get(site_call, 
                                headers={"X-API-Key":self.api_key})
        return int(request.json()['Response']['bungieNetUser']['membershipId'])
    
    def get_DestinyUserProfile(self, membership_id, components=[100]):
        '''
        Use new Destiny 2 endpoint for PSN player using the Destiny membershipId 
        '''
        components = "?components=" + ','.join([str(c) for c in components])
        site_call = "https://bungie.net/Platform/Destiny2/2/Profile/" + str(membership_id) + "/" + components
        request = requests.get(site_call, 
                                headers={"X-API-Key":self.api_key})
        return request.json()
    
    def get_postGameStats(self, game_id):
        site_call = "https://bungie.net/Platform/Destiny2/Stats/PostGameCarnageReport/" + str(game_id)
        request = requests.get(site_call, 
                                headers={"X-API-Key":self.api_key})
        return request.json()
    



In [182]:
bungie = BungieData(api_key=os.environ["BUNGIE_API_KEY"]) # Never put your keys in code... export 'em!

In [168]:
# Get my own Bungie MembershipID
Me = bungie.get_playerByTagName("BalancedSeeker6")
Me

{u'ErrorCode': 1,
 u'ErrorStatus': u'Success',
 u'Message': u'Ok',
 u'MessageData': {},
 u'Response': [{u'displayName': u'BalancedSeeker6',
   u'iconPath': u'/img/theme/destiny/icons/icon_psn.png',
   u'membershipId': u'4611686018453385415',
   u'membershipType': 2}],
 u'ThrottleSeconds': 0}

In [169]:
# Get my BUNGIE membershipID from using my DESTINY membershipID found in the above call
# This is the number found on the profile page of a user on Bungie.net
my_bungie_id = bungie.get_BungieUserId(4611686018453385415)
my_bungie_id

11927477

### To use GetProfile, you'll need to lookup the user's Destiny membershipID by searching by gamertag. Found that in the first call above: 4611686018453385415

In [183]:
My_Profile = bungie.get_DestinyUserProfile(4611686018453385415, components=[100,200])
My_Profile 

{u'ErrorCode': 1,
 u'ErrorStatus': u'Success',
 u'Message': u'Ok',
 u'MessageData': {},
 u'Response': {u'characters': {u'data': {u'2305843009260518141': {u'baseCharacterLevel': 20,
     u'characterId': u'2305843009260518141',
     u'classHash': 3655393761,
     u'classType': 0,
     u'dateLastPlayed': u'2017-09-30T07:06:14Z',
     u'emblemBackgroundPath': u'/common/destiny2_content/icons/c06afc4c698fbec7ee17e9d21496e192.jpg',
     u'emblemHash': 4077939641,
     u'emblemPath': u'/common/destiny2_content/icons/60530e4d21f5a32bebdca82cb178c4bf.jpg',
     u'genderHash': 2204441813,
     u'genderType': 1,
     u'levelProgression': {u'currentProgress': 146000,
      u'dailyLimit': 0,
      u'dailyProgress': 0,
      u'level': 20,
      u'levelCap': 20,
      u'nextLevelAt': 0,
      u'progressToNextLevel': 0,
      u'progressionHash': 1716568313,
      u'stepIndex': 20,
      u'weeklyLimit': 0,
      u'weeklyProgress': 0},
     u'light': 283,
     u'membershipId': u'4611686018453385415',
  

In [171]:
# Get a random single game's stats
game_stats = bungie.get_postGameStats(100)
game_stats

{u'ErrorCode': 1,
 u'ErrorStatus': u'Success',
 u'Message': u'Ok',
 u'MessageData': {},
 u'Response': {u'activityDetails': {u'directorActivityHash': 1658347443,
   u'instanceId': u'100',
   u'isPrivate': False,
   u'mode': 2,
   u'modes': [7, 2],
   u'referenceId': 1658347443},
  u'entries': [{u'characterId': u'2305843009260533952',
    u'extended': {u'values': {u'precisionKills': {u'basic': {u'displayValue': u'55',
        u'value': 55.0}}},
     u'weapons': [{u'referenceId': 53159280,
       u'values': {u'uniqueWeaponKills': {u'basic': {u'displayValue': u'4',
          u'value': 4.0}},
        u'uniqueWeaponKillsPrecisionKills': {u'basic': {u'displayValue': u'75%',
          u'value': 0.75}},
        u'uniqueWeaponPrecisionKills': {u'basic': {u'displayValue': u'3',
          u'value': 3.0}}}},
      {u'referenceId': 4024037919,
       u'values': {u'uniqueWeaponKills': {u'basic': {u'displayValue': u'75',
          u'value': 75.0}},
        u'uniqueWeaponKillsPrecisionKills': {u'basic'